In [285]:
import glob
import numpy as np
import random
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

import keras
from keras.layers import LSTM, Dense, Dropout, Flatten, Conv1D, MaxPooling2D, Bidirectional
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [286]:
SEED = 2017
DATA_DIR = 'Data/spoken_numbers_pcm/'

In [287]:
files = glob.glob(DATA_DIR + "*.wav")
X_train, X_val = train_test_split(files, test_size=0.2, random_state=SEED)

print('# Training examples: {}'.format(len(X_train)))
print('# Validation examples: {}'.format(len(X_val)))

# Training examples: 168
# Validation examples: 42


In [288]:
labels = []
for i in range(len(X_train)):
    label = X_train[i].split('/')[-1].split('_')[-2]
    if label not in labels:
        labels.append(label)
print(labels)

['farae', 'umair', 'hamza']


In [289]:
label_binarizer = LabelBinarizer()
label_binarizer.fit(list(set(labels)))

def one_hot_encode(x): return label_binarizer.transform(x)

In [290]:
n_features = 20
max_length = 100
n_classes = len(labels)

In [291]:
def batch_generator(data, batch_size=16):
    while 1:
        random.shuffle(data)
        X, y = [], []
        for i in range(batch_size):
            wav = data[i]
            wave, sr = librosa.load(wav, mono=True)
            label = wav.split('/')[-1].split('_')[-2]
            y.append(label)
            mfcc = librosa.feature.mfcc(y=wave, sr=sr)
            #print(len(mfcc[0]))
            mfcc = np.pad(mfcc, ((0,0), (0, max_length-len(mfcc[0]))), mode='constant', constant_values=0) 
            X.append(np.array(mfcc))
        yield np.array(X), np.array(one_hot_encode(y))

In [292]:
learning_rate = 0.000001
batch_size = 4
n_epochs = 50
dropout = 0.5

input_shape = (n_features, max_length)
steps_per_epoch = 50

In [293]:
#model = Sequential()
#model.add(LSTM(256, return_sequences=True, input_shape=input_shape, dropout=dropout))
#model.add(Flatten())
#model.add(Dense(128, activation='relu'))
#model.add(Dropout(dropout))
#model.add(Dense(n_classes, activation='softmax'))

In [302]:
model = Sequential()
model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(None, 100)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))

In [303]:
opt = Adam(learning_rate=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_4 (Bidirectio  (None, None, 256)        234496    
 nal)                                                            
                                                                 
 dropout_32 (Dropout)        (None, None, 256)         0         
                                                                 
 bidirectional_5 (Bidirectio  (None, 128)              164352    
 nal)                                                            
                                                                 
 dropout_33 (Dropout)        (None, 128)               0         
                                                                 
 dense_60 (Dense)            (None, 3)                 387       
                                                                 
Total params: 399,235
Trainable params: 399,235
Non-t

In [304]:
callbacks = [ModelCheckpoint('checkpoints/voice_recognition_best_model_{epoch:02d}.hdf5', save_best_only=True),
            EarlyStopping(monitor='val_loss', mode='min', restore_best_weights=True, patience=5)]

In [305]:
history = model.fit(
 batch_generator(X_train, batch_size),
 steps_per_epoch=steps_per_epoch,
 epochs=n_epochs,
 verbose=1,
 validation_data=batch_generator(X_val, 2),
 validation_steps=5,
 callbacks=callbacks
 )

Epoch 1/50
50/50 [==============================] - 7s 62ms/step - loss: 0.9962 - accuracy: 0.5250 - val_loss: 1.1387 - val_accuracy: 0.2000
Epoch 2/50
50/50 [==============================] - 2s 43ms/step - loss: 1.0065 - accuracy: 0.4500 - val_loss: 1.1583 - val_accuracy: 0.2000
Epoch 3/50
50/50 [==============================] - 2s 43ms/step - loss: 0.9803 - accuracy: 0.5550 - val_loss: 1.0946 - val_accuracy: 0.4000
Epoch 4/50
50/50 [==============================] - 2s 43ms/step - loss: 0.9430 - accuracy: 0.6400 - val_loss: 0.9808 - val_accuracy: 0.5000
Epoch 5/50
50/50 [==============================] - 2s 42ms/step - loss: 0.9751 - accuracy: 0.5700 - val_loss: 1.0312 - val_accuracy: 0.6000
Epoch 6/50
50/50 [==============================] - 2s 42ms/step - loss: 0.9305 - accuracy: 0.6300 - val_loss: 1.0374 - val_accuracy: 0.3000
Epoch 7/50
50/50 [==============================] - 2s 42ms/step - loss: 0.9402 - accuracy: 0.5800 - val_loss: 0.8859 - val_accuracy: 0.7000
Epoch 8/50
50

In [306]:
X_train[0].split('/')[-1].split('_')[-2]

'farae'